# Imports


In [2]:
import pandas as pd
import os
%matplotlib inline 

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import style
style.use('ggplot')

from scipy import stats

from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.multioutput import MultiOutputClassifier

from sklearn.model_selection import cross_val_score

# Read data

In [2]:
# import opendatasets as od
# dataset = 'https://www.kaggle.com/competitions/child-mind-institute-problematic-internet-use/data'
# od.download(dataset)

In [3]:
df = pd.read_csv("/home/ku18/CS412/dataset/train.csv")
df.columns

Index(['id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
       'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST'

In [4]:
TARGET_COLS = [
    # "PCIAT-PCIAT_01",
    # "PCIAT-PCIAT_02",
    # "PCIAT-PCIAT_03",
    # "PCIAT-PCIAT_04",
    # "PCIAT-PCIAT_05",
    # "PCIAT-PCIAT_06",
    # "PCIAT-PCIAT_07",
    # "PCIAT-PCIAT_08",
    # "PCIAT-PCIAT_09",
    # "PCIAT-PCIAT_10",
    # "PCIAT-PCIAT_11",
    # "PCIAT-PCIAT_12",
    # "PCIAT-PCIAT_13",
    # "PCIAT-PCIAT_14",
    # "PCIAT-PCIAT_15",
    # "PCIAT-PCIAT_16",
    # "PCIAT-PCIAT_17",
    # "PCIAT-PCIAT_18",
    # "PCIAT-PCIAT_19",
    # "PCIAT-PCIAT_20",
    # "PCIAT-PCIAT_Total",
    "sii"
]

FEATURE_COLS = [
    "Basic_Demos-Age",
    "Basic_Demos-Sex",
    "CGAS-CGAS_Score",
    "Physical-BMI",
    "Physical-Height",
    "Physical-Weight",
    "Physical-Waist_Circumference",
    "Physical-Diastolic_BP",
    "Physical-HeartRate",
    "Physical-Systolic_BP",
    "Fitness_Endurance-Max_Stage",
    "Fitness_Endurance-Time_Mins",
    "Fitness_Endurance-Time_Sec",
    "FGC-FGC_CU",
    "FGC-FGC_CU_Zone",
    "FGC-FGC_GSND",
    "FGC-FGC_GSND_Zone",
    "FGC-FGC_GSD",
    "FGC-FGC_GSD_Zone",
    "FGC-FGC_PU",
    "FGC-FGC_PU_Zone",
    "FGC-FGC_SRL",
    "FGC-FGC_SRL_Zone",
    "FGC-FGC_SRR",
    "FGC-FGC_SRR_Zone",
    "FGC-FGC_TL",
    "FGC-FGC_TL_Zone",
    "BIA-BIA_Activity_Level_num",
    "BIA-BIA_BMC",
    "BIA-BIA_BMI",
    "BIA-BIA_BMR",
    "BIA-BIA_DEE",
    "BIA-BIA_ECW",
    "BIA-BIA_FFM",
    "BIA-BIA_FFMI",
    "BIA-BIA_FMI",
    "BIA-BIA_Fat",
    "BIA-BIA_Frame_num",
    "BIA-BIA_ICW",
    "BIA-BIA_LDM",
    "BIA-BIA_LST",
    "BIA-BIA_SMM",
    "BIA-BIA_TBW",
    "PAQ_A-PAQ_A_Total",
    "PAQ_C-PAQ_C_Total",
    "SDS-SDS_Total_Raw",
    "SDS-SDS_Total_T",
    "PreInt_EduHx-computerinternet_hoursday",
]
data = df[FEATURE_COLS]
target = df[TARGET_COLS]

In [5]:
len(data)

3960

# Missing values in data

In [6]:
print(data.isna().sum())

Basic_Demos-Age                              0
Basic_Demos-Sex                              0
CGAS-CGAS_Score                           1539
Physical-BMI                               938
Physical-Height                            933
Physical-Weight                            884
Physical-Waist_Circumference              3062
Physical-Diastolic_BP                     1006
Physical-HeartRate                         993
Physical-Systolic_BP                      1006
Fitness_Endurance-Max_Stage               3217
Fitness_Endurance-Time_Mins               3220
Fitness_Endurance-Time_Sec                3220
FGC-FGC_CU                                1638
FGC-FGC_CU_Zone                           1678
FGC-FGC_GSND                              2886
FGC-FGC_GSND_Zone                         2898
FGC-FGC_GSD                               2886
FGC-FGC_GSD_Zone                          2897
FGC-FGC_PU                                1650
FGC-FGC_PU_Zone                           1689
FGC-FGC_SRL  

## Iterative imputation to fill in missing values in the dataset|

In [7]:
from sklearn.experimental import enable_iterative_imputer 
from sklearn.impute import IterativeImputer

iterative_imputer = IterativeImputer(max_iter=10, random_state=0)

data_imputed = pd.DataFrame(iterative_imputer.fit_transform(data))
data_imputed.columns = FEATURE_COLS

# Missing values in target

In [8]:
print(target.isna().sum())

sii    1224
dtype: int64


## Label Propogation to fill target values

In [9]:
from sklearn.semi_supervised import LabelPropagation
from sklearn.preprocessing import StandardScaler
target = target.fillna(-1).values.flatten()
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)
label_prop_model = LabelPropagation(kernel='knn', n_neighbors=5)
label_prop_model.fit(data_scaled, target)
target_imputed = label_prop_model.transduction_ 

# Classification models

In [10]:
X = data_imputed
y = target_imputed

## Classifiers

In [11]:
# create the model: Random Forest
myRF = RandomForestClassifier(max_depth=15, random_state=0, n_estimators=100)

# Perform k-fold cross-validation
accuracyRF = cross_val_score(myRF,X,y,cv=10)

# report accuracy in each fold, and mean accuracy
print(accuracyRF)
print(np.mean(accuracyRF))

[0.67929293 0.69191919 0.68434343 0.69949495 0.67424242 0.67171717
 0.67929293 0.7020202  0.6540404  0.66161616]
0.6797979797979798


In [22]:
# create the model: XGBoost
myXGB = XGBClassifier(max_depth=15, n_estimators = 100)

# Perform k-fold cross-validation and report accuracy in each fold
accuracyXGB = cross_val_score(myXGB, X, y, cv=10)
print("Cross-validation accuracies for each fold:", accuracyXGB)
print("Mean cross-validation accuracy:", accuracyXGB.mean())

myXGB.fit(X, y)

Cross-validation accuracies for each fold: [0.66161616 0.67424242 0.67676768 0.68686869 0.67929293 0.68434343
 0.66919192 0.69444444 0.63636364 0.7020202 ]
Mean cross-validation accuracy: 0.6765151515151515


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=15, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [37]:
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

mySVC = make_pipeline(StandardScaler(), SVC(kernel='linear', random_state=42))
accuracySVC = cross_val_score(mySVC,X,y,cv=10)
print(accuracySVC)
print(np.mean(accuracySVC))

[0.6489899  0.65656566 0.64646465 0.6489899  0.6489899  0.64646465
 0.6489899  0.6489899  0.6540404  0.65151515]
0.65


In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# Standardize features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Convert data to PyTorch tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Create data loaders
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# Define the Feedforward Neural Network Model
class FFN(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(FFN, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = nn.Linear(hidden_dim, output_dim)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        x = self.relu(x)
        x = self.fc3(x)
        return self.softmax(x)  # For prediction, softmax is used to output class probabilities

# Instantiate the model
input_dim = X_train.shape[1]
hidden_dim = 64  # Example hidden layer size, adjust as needed
output_dim = len(set(y))  # Number of classes
model = FFN(input_dim, hidden_dim, output_dim)

# Define Loss and Optimizer
criterion = nn.CrossEntropyLoss()  # Suitable for multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [42]:
from sklearn.model_selection import KFold
import numpy as np

# Define 10-fold cross-validation
kf = KFold(n_splits=10, shuffle=True, random_state=42)
num_epochs = 20  # Number of epochs to train for each fold
batch_size = 32  # Batch size for DataLoader
fold_accuracies = []  # To store accuracy for each fold

# Loop over each fold
for fold, (train_idx, val_idx) in enumerate(kf.split(X, y)):
    print(f"Fold {fold+1}")

    # Split data into train and validation sets for this fold
    # X_train_fold, X_val_fold = X[train_idx], X[val_idx]
    y_train_fold, y_val_fold = y[train_idx], y[val_idx]
    X_train_fold, X_val_fold = X.iloc[train_idx], X.iloc[val_idx]
    # y_train_fold, y_val_fold = y.iloc[train_idx], y.iloc[val_idx]

    # Standardize features
    scaler = StandardScaler()
    X_train_fold = scaler.fit_transform(X_train_fold)
    X_val_fold = scaler.transform(X_val_fold)

    # Convert to PyTorch tensors
    X_train_tensor = torch.tensor(X_train_fold, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train_fold, dtype=torch.long)
    X_val_tensor = torch.tensor(X_val_fold, dtype=torch.float32)
    y_val_tensor = torch.tensor(y_val_fold, dtype=torch.long)

    # Create data loaders
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)

    # Initialize a new instance of the model for each fold
    model = FFN(input_dim, hidden_dim, output_dim)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    
    # Train the model
    for epoch in range(num_epochs):
        model.train()
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

    # Evaluate the model on the validation set
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            total += y_batch.size(0)
            correct += (predicted == y_batch).sum().item()

    # Calculate accuracy for this fold
    fold_accuracy = correct / total
    fold_accuracies.append(fold_accuracy)
    print(f"Accuracy for fold {fold+1}: {fold_accuracy:.4f}")

# Report mean and standard deviation of accuracy across all folds
print("Cross-validation results:")
print(f"Mean accuracy: {np.mean(fold_accuracies):.4f}")
print(f"Standard deviation of accuracy: {np.std(fold_accuracies):.4f}")


Fold 1
Accuracy for fold 1: 0.7096
Fold 2
Accuracy for fold 2: 0.6490
Fold 3
Accuracy for fold 3: 0.6692
Fold 4
Accuracy for fold 4: 0.6919
Fold 5
Accuracy for fold 5: 0.6894
Fold 6
Accuracy for fold 6: 0.6843
Fold 7
Accuracy for fold 7: 0.6869
Fold 8
Accuracy for fold 8: 0.6742
Fold 9
Accuracy for fold 9: 0.6490
Fold 10
Accuracy for fold 10: 0.6944
Cross-validation results:
Mean accuracy: 0.6798
Standard deviation of accuracy: 0.0186


# test.csv performance

In [35]:
df = pd.read_csv("/home/ku18/CS412/dataset/test.csv")
df.columns

Index(['id', 'Basic_Demos-Enroll_Season', 'Basic_Demos-Age', 'Basic_Demos-Sex',
       'CGAS-Season', 'CGAS-CGAS_Score', 'Physical-Season', 'Physical-BMI',
       'Physical-Height', 'Physical-Weight', 'Physical-Waist_Circumference',
       'Physical-Diastolic_BP', 'Physical-HeartRate', 'Physical-Systolic_BP',
       'Fitness_Endurance-Season', 'Fitness_Endurance-Max_Stage',
       'Fitness_Endurance-Time_Mins', 'Fitness_Endurance-Time_Sec',
       'FGC-Season', 'FGC-FGC_CU', 'FGC-FGC_CU_Zone', 'FGC-FGC_GSND',
       'FGC-FGC_GSND_Zone', 'FGC-FGC_GSD', 'FGC-FGC_GSD_Zone', 'FGC-FGC_PU',
       'FGC-FGC_PU_Zone', 'FGC-FGC_SRL', 'FGC-FGC_SRL_Zone', 'FGC-FGC_SRR',
       'FGC-FGC_SRR_Zone', 'FGC-FGC_TL', 'FGC-FGC_TL_Zone', 'BIA-Season',
       'BIA-BIA_Activity_Level_num', 'BIA-BIA_BMC', 'BIA-BIA_BMI',
       'BIA-BIA_BMR', 'BIA-BIA_DEE', 'BIA-BIA_ECW', 'BIA-BIA_FFM',
       'BIA-BIA_FFMI', 'BIA-BIA_FMI', 'BIA-BIA_Fat', 'BIA-BIA_Frame_num',
       'BIA-BIA_ICW', 'BIA-BIA_LDM', 'BIA-BIA_LST'

In [36]:
ids = df['id']
test_data = df[FEATURE_COLS]

In [37]:
len(test_data)

20

In [38]:
test_data.isna().sum()

Basic_Demos-Age                            0
Basic_Demos-Sex                            0
CGAS-CGAS_Score                           12
Physical-BMI                               7
Physical-Height                            7
Physical-Weight                            7
Physical-Waist_Circumference              15
Physical-Diastolic_BP                      9
Physical-HeartRate                         8
Physical-Systolic_BP                       9
Fitness_Endurance-Max_Stage               17
Fitness_Endurance-Time_Mins               17
Fitness_Endurance-Time_Sec                17
FGC-FGC_CU                                 7
FGC-FGC_CU_Zone                            7
FGC-FGC_GSND                              15
FGC-FGC_GSND_Zone                         15
FGC-FGC_GSD                               15
FGC-FGC_GSD_Zone                          15
FGC-FGC_PU                                 7
FGC-FGC_PU_Zone                            7
FGC-FGC_SRL                                7
FGC-FGC_SR

In [39]:
# Using the trained Iterative Imputer fill in missing values in test
test_data_imputed = pd.DataFrame(iterative_imputer.fit_transform(test_data))
test_data_imputed.columns = FEATURE_COLS

/opt/conda/lib/python3.11/site-packages/sklearn/impute/_iterative.py:800: ConvergenceWarning: [IterativeImputer] Early stopping criterion not reached.
  warnings.warn(


In [40]:
test_predictions = myXGB.predict(test_data_imputed)

In [41]:
test_predictions

array([0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0, 0, 0, 0])

In [44]:
result_df = pd.DataFrame({
    'id': ids[0],  # Assuming 'id' is the column name for IDs
    'sii': test_predictions
})

In [46]:
result_df.to_csv("submission.csv")